# 3. Issue Dianosis: Agent Bricks Custom LLM

Data Flow: raw reviews -> review aspect extractions -> location aspect daily -> flag all issues -> **issue diagnosis and recommendations**


## Build Custom LLM Agent (Summary + Diagnosis)

- Instructions: 
```
You are an experienced hotel operations analyst.
Given a single issue aspect (for example, “cleanliness” or “temperature_hvac”) and several short review extracts related to that issue, write a concise and factual diagnosis summarizing what the data indicates.

Rules:
	1.	Base everything strictly on the provided reviews. Identify common complaints and patterns across guests.
	2.	Do not make up facts — only infer root causes that clearly align with evidence.
	3.	Keep tone factual, professional, and concise.
	4.	Summarize in the following four parts only:
		• issue_summary → what the guests are saying (pattern)
		• potential_root_cause → why it’s likely happening
		• impact → how it affects guest experience or brand
		• recommended_action → clear, actionable steps to fix or prevent recurrence
	5.	The output must be valid JSON. No markdown, no additional commentary.
	6.	Keep total text per field under ~3 sentences.

Output format:
{
  "aspect": "<aspect_name>",
  "issue_summary": "<short summary of the pattern in guest reviews>",
  "potential_root_cause": "<likely operational or maintenance causes>",
  "impact": "<how it affects guest satisfaction, comfort, or reputation>",
  "recommended_action": "<specific and practical steps to address the issue>"
}
```

## Batch Inference With CLLM Agent Endpoint and `AI_QUERY`
[CLLM Query in SQL Editor](/Workspace/Users/cindy.wu@databricks.com/voc_industry_demo/queries/AI Query with CLLM Diagnosis.dbquery.ipynb)

In [0]:
%sql
CREATE OR REPLACE TABLE lakehouse_inn_catalog.voc.open_issues_diagnosis AS
WITH query_results AS (
  SELECT
    *,
    ai_query(
      't2t-e71a8879-endpoint',
      relevant_reviews_text,
      failOnError => false
    ) AS response
  FROM (
    SELECT *
    FROM lakehouse_inn_catalog.voc.issues_daily
    -- WHERE status = 'Open'
  )
)
SELECT
  -- Exclude the response column
  issue_id,
  opened_at,
  location,
  aspect,
  severity,
  status,
  open_reason,
  delta_pp_open,
  nms_open,
  volume_open,
  relevant_reviews,
  response.result AS response_result,
  response.errorMessage AS response_error,
  current_date() AS processed_date
FROM query_results;



In [0]:
display(spark.table('lakehouse_inn_catalog.voc.open_issues_diagnosis'))

## Next Step: AI/BI Dashboard & Genie Room

## MAS Agent with KA and Genie Room

0. data generation
- generate raw review data with ai_query based on list of locations and channels
- generate hotel runbook based on list of aspects
- [Lakebase] Sync Hotel runbook to lakebase

1. Extract Insights
- [Agent Bricks, SQL] Build Infmroation Extraction Agent and AI_QUERY to extract insights from raw review 

2. Diagnosis
- [Notebook] Aggreate insights by location-aspect and calcualte metrics
- [Notebook] Create issues table based on extracted location-aspect metrics
- [Lakebass] Sync latest issues table to lakebase
- [Agent Bricks] Use Custom LLM to idenitify issue causes based on relevant reviews and provide summary
- [Lakebase, Apps] Sync issues table to lakebase
- [AI/BI] Build Genie room for raw reviews and issues table for deep dive
- [AI/BI, Apps] Build dashboard to visualize issues by location, map view

3. Recommendations
- [Notebook] Generate emails based on issue and hotel runbook in App
- [Agent Bricks, Apps] Build MAS with Genie and KA 


In [0]:
from pyspark.sql import functions as F, Window as W
diag = "lakehouse_inn_catalog.voc.open_issues_diagnosis"

# Load issues table
issues_df = spark.table(diag)

# Window to get the latest opened_at per (aspect, location)
w = W.partitionBy("aspect", "location").orderBy(F.col("opened_at").desc())

# Add row number to identify the latest issue per group
issues_ranked = issues_df.withColumn("rn", F.row_number().over(w))

# Create new status column: only the latest as Open, others as Closed
issues_final = (
    issues_ranked.withColumn(
        "status",
        F.when(F.col("rn") == 1, F.lit("Open")).otherwise(F.lit("Closed"))
    )
    .drop("rn")
)

# Overwrite the table with the updated status column
issues_final.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(diag)